<a href="https://colab.research.google.com/github/AndreiS22/deep_learning_labs/blob/main/docs/labs/lab4/4_4_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 4: Using GPU acceleration with PyTorch

In [2]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try:
    import torchbearer
except:
    !pip install torchbearer

## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [3]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-5.4443e-01, -6.4965e-01,  1.9436e+00,  ...,  2.4597e-02,
         -4.7167e-01, -2.8388e-01],
        [-1.9205e-01,  1.9058e+00,  8.3493e-02,  ..., -7.5319e-01,
          5.3207e-01, -2.4247e-01],
        [ 7.5848e-01, -1.9603e+00,  1.0608e+00,  ..., -3.6558e-03,
         -1.8274e+00,  6.1799e-02],
        ...,
        [-1.5272e+00,  3.0820e-01,  9.3823e-02,  ..., -1.3559e-01,
          1.4890e+00, -1.2641e+00],
        [ 5.1616e-02, -5.6298e-01, -4.2088e-01,  ..., -7.4152e-01,
         -1.2621e-01,  4.1961e+00],
        [-3.1228e-02, -1.1171e-02,  1.8382e-01,  ..., -7.2050e-01,
         -7.9725e-01, -6.5224e-01]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [4]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-3.7341e-01, -3.9887e-01,  1.3427e-02,  ..., -1.8260e+00,
         -1.0715e+00, -5.9938e-01],
        [-3.2561e-03,  1.1420e+00, -1.5551e+00,  ..., -1.3068e-02,
          5.0884e-02,  3.7446e-01],
        [ 5.3728e-01,  2.7433e-02,  2.0229e+00,  ..., -1.0066e-03,
         -2.6441e-02,  5.1788e-01],
        ...,
        [ 1.6120e+00, -2.8190e-01, -1.8552e+00,  ...,  1.0860e+00,
         -1.6561e-01,  2.2739e-01],
        [ 9.9027e-01,  4.7079e-02, -9.7564e-02,  ...,  6.6349e-02,
          5.0205e-02, -2.5373e-02],
        [ 2.1773e-01,  5.8149e-01,  3.8787e-01,  ..., -3.4849e-01,
          3.7321e-01, -5.5146e-01]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[ 1.2224e-01, -1.6112e-01, -3.8362e-01,  ...,  3.0394e-01,
          1.2968e-01, -1.5633e-01],
        [ 2.9557e-01,  9.9558e-02, -2.0798e-02,  ...,  3.0400e-01,
          4.0794e-02,  3.6958e-02],
        [ 2.6261e-03, -7.0229e-01,  3.4854e-01,  ...,  1.1339e-01,
         -7.5166e-01, -1.0334e+00],
        ...,
        [ 1.1954e+00, -1.4740e-01,  1.6580e+00,  ..., -5.9789e-01,
         -1.1143e+00, -4.2827e-01],
        [-2.2236e-01, -2.9960e-01, -1.3017e-01,  ..., -9.5086e-02,
         -2.3263e-01,  1.5251e-01],
        [-2.9106e-01,  3.0746e+00,  9.4663e-02,  ..., -5.1480e+00,
          7.2018e-01,  1.3367e-01]], device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

Parallelisation of mathematical operations through linear algebra

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

Limited memory available to store and train the entire models.
